In [2]:
import pandas as pd
import os
import numpy as np
import datetime
from datetime import timedelta
import fnmatch

In [2]:
## As what greg did###
#1. Load an allied health file
#2. Ensure allied health file as examdate
#3. Do a right type join (assume left = og file, right = allied health file)
#4. Extract rows only with a T1 file path, so we have full rows
#5. Calculate abs(allied health data - MRI visit date) and label it like df.datediffs or some shit
#6. df.groupby with PTID, and then sort by datediffs ascend
#7. Keep the row where datadiff is less than 28 days and the smallest val. If tie, then we ball and just choose at random
#8. Conduct a final length of rows check - MUST BE 15733
#9. Rinse and repeat

In [15]:
df = pd.read_csv('/rds/general/project/c3nl_scott_students/ephemeral/sankeith/gregbased_scott10k_alliedhealth.csv', low_memory = False)
df[f'VISDATE'] = pd.to_datetime(df[f'VISDATE'])
df[f'EXAMDATE_4WKS_LATER'] = pd.to_datetime(df[f'EXAMDATE_4WKS_LATER'])
df[f'EXAMDATE_4WKS_B4'] = pd.to_datetime(df[f'EXAMDATE_4WKS_B4'])
print(len(df))
print(len(df.columns))
print(len(df['PTID'].unique()))

15733
14
2635


In [16]:
vars = [
    ['/rds/general/project/c3nl_scott_students/live/sankeith/scott_10k_housekeeping/scott_10k_extra_adni_info/DXSUM_10Sep2025.csv',['PTID','EXAMDATE','PHASE', 'DIAGNOSIS'], {'PHASE':np.nan,'DIAGNOSIS':np.nan}],
    ['/rds/general/project/c3nl_scott_students/live/sankeith/scott_10k_housekeeping/scott_10k_extra_adni_info/MMSE_28Mar2025.csv', ['PTID','VISDATE','MMSCORE'], {'MMSCORE':np.nan}],
    ['/rds/general/project/c3nl_scott_students/live/sankeith/scott_10k_housekeeping/scott_10k_extra_adni_info/GDSCALE_28Mar2025.csv',['PTID','VISDATE','GDTOTAL'], {'GDTOTAL':np.nan}],
    ['/rds/general/project/c3nl_scott_students/live/sankeith/scott_10k_housekeeping/scott_10k_extra_adni_info/MOCA_30Mar2025.csv', ['PTID','VISDATE', 'MOCA'], {'MOCA':np.nan}],
    ['/rds/general/project/c3nl_scott_students/live/sankeith/scott_10k_housekeeping/scott_10k_extra_adni_info/NEUROBAT_28Mar2025.csv', ['PTID','VISDATE','LIMMTOTAL', 'CLOCKSCOR', 'LDELTOTAL', 'LDELCUE', 'ANART'], {'LIMMTOTAL':np.nan, 'CLOCKSCOR':np.nan, 'LDELTOTAL':np.nan, 'LDELCUE':np.nan, 'ANART':np.nan}]
]
for i in range(0,len(vars)):
    column_suffix = vars[i][0].split('/')[-1]
    column_suffix = column_suffix.split('_')[0]
    varnames = vars[i][1]
    
    #Load an allied health file
    Talli = pd.read_csv(vars[i][0], low_memory = False)
    Talli = Talli[vars[i][1]]
    
    if 'RID' in Talli.columns:
        Talli.drop(columns = 'RID', inplace = True)    
    print(f"Length of {column_suffix}: {len(Talli)}")
    
    #Ensure allied health file as examdate
    if 'VISDATE' in Talli.columns:
        Talli.rename(columns = {'VISDATE':f"EXAMDATE_{column_suffix}"}, inplace = True)
        Talli[f'EXAMDATE_{column_suffix}'] = pd.to_datetime(Talli[f'EXAMDATE_{column_suffix}'], errors = 'coerce')
    else:
        Talli[f'EXAMDATE'] = pd.to_datetime(Talli[f'EXAMDATE'], errors = 'coerce')

    #Do a right type join
    Tsub = pd.merge(df,Talli, on = 'PTID', how = 'left')
    print(f"Length of the freshly joined dataframe: {len(Tsub)}")
    
    #Extract rows only with a T1 file path, so we have full rows
    Tsub.dropna(subset='T1_PATH',inplace=True)
    print(f"After dropping rows without T1 paths: {len(Tsub)}")

    #Calculate abs(allied health data - MRI visit date) and label it like df.datediffs or some shit
    try:
        Tsub[f'DATEDIFFS_{column_suffix}'] = abs(Tsub[f'EXAMDATE_{column_suffix}'] - Tsub['VISDATE'])
    except KeyError:
        Tsub[f'DATEDIFFS_{column_suffix}'] = abs(Tsub['EXAMDATE'] - Tsub['VISDATE'])

    #df.groupby with PTID, and then sort by datediffs ascend. Select the/a row where DATEDIFFS is the smallest
    Tsub = Tsub.sort_values(f'DATEDIFFS_{column_suffix}', ascending=True)
    
    min_diffs = (
    Tsub
    .groupby('T1_PATH')[f'DATEDIFFS_{column_suffix}']
    .transform('min')
    )
    
    Tsub = Tsub[
    (Tsub[f'DATEDIFFS_{column_suffix}'] == min_diffs) |
    (min_diffs.isna() & Tsub[f'DATEDIFFS_{column_suffix}'].isna())
    ]

    Tsub.reset_index(drop=True, inplace=True)

    mask = Tsub[f'DATEDIFFS_{column_suffix}'] > pd.Timedelta(28, "D")
    
    Tsub.loc[mask, f'DATEDIFFS_{column_suffix}'] = pd.NaT
    
    for key in vars[i][2]:
        Tsub.loc[mask, key] = vars[i][2].get(key)

    print(len(Tsub))
    print(type(Tsub))
    Tsub.drop_duplicates(keep = 'first',inplace = True)
    print(len(Tsub))
    print(Tsub[f'DATEDIFFS_{column_suffix}'].describe())
    
    dupe_mask = Tsub.duplicated(subset='T1_PATH', keep=False)
    Tsub_dupes = Tsub.loc[dupe_mask].copy()
    print(Tsub_dupes)
    Tsub_dupes.to_csv(f'/rds/general/project/c3nl_scott_students/ephemeral/sankeith/final_gregbased_scott10k_alliedhealth_{i}.csv', index = False)


    Tsub.to_csv('/rds/general/project/c3nl_scott_students/ephemeral/sankeith/final_gregbased_scott10k_alliedhealth.csv', index = False)
    break


Length of DXSUM: 15674
Length of the freshly joined dataframe: 112212
After dropping rows without T1 paths: 112212
15811
<class 'pandas.core.frame.DataFrame'>
15772
count                        11967
mean     8 days 03:34:47.440461268
std      7 days 19:34:22.459119784
min                0 days 00:00:00
25%                1 days 00:00:00
50%                6 days 00:00:00
75%               14 days 00:00:00
max               28 days 00:00:00
Name: DATEDIFFS_DXSUM, dtype: object
             PTID     RID                                  DATA_KEY  T1_YEAR  \
938    123_S_0108   108.0  123_S_0108_ADNI-T1_2011-02-15_08_58_12.0   2011.0   
939    123_S_0108   108.0  123_S_0108_ADNI-T1_2011-02-15_08_58_12.0   2011.0   
2994   128_S_1407  1407.0  128_S_1407_ADNI-T1_2012-01-03_15_26_25.0   2012.0   
2995   128_S_1407  1407.0  128_S_1407_ADNI-T1_2012-01-03_15_26_25.0   2012.0   
3006   128_S_1407  1407.0  128_S_1407_ADNI-T1_2012-01-03_15_18_42.0   2012.0   
...           ...     ...             

2635